# Текущий дневной объём аудиозаписей для расшифровки

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected"
SEED = 5908

In [2]:
trs = pd.read_csv(
    'transcribations_2024.csv',
    dtype={19: object},
    parse_dates= [1, 17, 23],
    index_col=0
)

In [3]:
display(
    trs.info(),
    trs.describe().style.format('{:.2f}'),
    trs.sample(10, random_state=SEED).T
    )

<class 'pandas.core.frame.DataFrame'>
Index: 3357462 entries, 0 to 3357461
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   transcribation_date  datetime64[ns]
 1   date_y               float64       
 2   date_m               float64       
 3   date_d               float64       
 4   side                 bool          
 5   text                 object        
 6   start                float64       
 7   audio_file_name      object        
 8   conf                 float64       
 9   end_time             float64       
 10  sentiment            float64       
 11  sentiment_pos        float64       
 12  sentiment_neg        float64       
 13  ID                   int64         
 14  linkedid             float64       
 15  dst                  object        
 16  record_date          datetime64[ns]
 17  source_id            int64         
 18  src                  object        
 19  cpu_id               int64

None

,transcribation_date,date_y,date_m,date_d,start,conf,end_time,sentiment,sentiment_pos,sentiment_neg,ID,linkedid,record_date,source_id,cpu_id,duration,file_size,queue_date
count,3357462.00,0.00,0.00,0.00,3357462.00,3357462.00,3357462.00,0.00,0.00,0.00,3357462.00,3356882.00,3357462.00,3357462.00,3357462.00,3357462.00,3357462.00,3357462.00
mean,.2f,nan,nan,nan,102.38,0.89,105.25,nan,nan,nan,167963000.50,1705127990.19,.2f,1.20,10.30,196.23,3139722.61,.2f
min,.2f,nan,nan,nan,0.00,0.00,0.00,nan,nan,nan,166284270.00,1704088914.15,.2f,1.00,0.00,0.00,0.00,.2f
25%,.2f,nan,nan,nan,28.35,0.87,31.11,nan,nan,nan,167123635.25,1704719688.15,.2f,1.00,4.00,85.16,1362604.00,.2f
50%,.2f,nan,nan,nan,68.43,0.95,71.46,nan,nan,nan,167963000.50,1705129009.16,.2f,1.00,9.00,165.74,2651884.00,.2f
75%,.2f,nan,nan,nan,138.69,1.00,141.66,nan,nan,nan,168802365.75,1705568808.24,.2f,1.00,16.00,251.86,4029804.00,.2f
max,.2f,nan,nan,nan,3592.83,1.00,3595.14,nan,nan,nan,169641731.00,1706016461.16,.2f,2.00,27.00,3599.08,57585324.00,.2f
std,nan,nan,nan,nan,117.48,0.19,117.80,nan,nan,nan,969215.94,521451.97,nan,0.40,7.71,172.36,2757835.52,nan


,684367,252012,9669,1590063,356555,3040087,191451,866631,2154132,2787414
transcribation_date,2024-01-07 13:51:19,2024-01-04 11:21:53,2024-01-02 09:52:01,2024-01-12 14:52:41,2024-01-04 19:35:41,2024-01-22 08:57:45,2024-01-03 16:19:18,2024-01-08 19:08:45,2024-01-16 11:43:15,2024-01-20 07:18:05
date_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date_m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date_d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
side,True,False,False,False,True,False,True,True,False,False
text,êàê ïîìî÷ü,à,îíî âàñ êàê èç öåëàÿ áëèíû èëè ó íå¸ ñòåêëîêåð...,àëëî çäðàâñòâóéòå ìàñòåð ïî òåëåâèçîðó,íó òàê äèàãíîñòèêó òî ïîëîìêó òî ïîñìîòðèòå,à ÷òî ïîëçóíêîâ ÷åãî çîâ¸ò êòî îñòàâëÿëè ÷òîáû...,ñíîâà âûêëþ÷àþò,äîáðûé âå÷åð,äîáðûé äåíü äåæóðíûé ìàñòåð âëàäèìèð àäëåð âû ...,ïðîâåðêà
start,26.43,26.37,40.71,11.94,398.43,12.72,100.508643,24.036608,14.85,6.39
audio_file_name,in_5164_2024-01-07-13-39-19rxtx-out.wav,in_9164467778_2024-01-04-11-09-11rxtx-in.wav,in_5063_2024-01-02-09-35-40rxtx-in.wav,a2024-01-12t14:41:23b_c9857207630d_e9252098540...,in_8127770011_2024-01-04-19-14-43rxtx-out.wav,a2024-01-22t08:41:33b_c9169035066d_e9166055729...,a2024-01-03t15:59:40b_c79032079978d_e912691150...,in_5124_2024-01-08-18-56-35rxtx-out.wav,a2024-01-16t11:30:31b_c9966816168d_e4997342649...,in_5077_2024-01-20-07-07-31rxtx-in.wav
conf,0.820503,0.867386,0.896242,1.0,0.998356,0.847019,1.0,0.841633,0.884995,0.90441
end_time,27.0,26.67,48.15,13.59,401.01,16.68,101.67,24.6,21.57,6.96


Удалим ненужные пустые колонки.

In [4]:
trs = trs.drop(
    ['date_y', 'date_m', 'sentiment', 'sentiment_pos', 'sentiment'],
    axis=1
)

Получим записи с уникальными именами файлов и ID разговора, чтобы учитывать длительность только уникальных незадублированных файлов. Оставим также только записи с распознанным текстом.

In [5]:
unique_files = trs[trs.text.notna()].drop_duplicates(
    subset=['linkedid', 'audio_file_name']
)
unique_files.date_d = unique_files.record_date.dt.date

Далее посчитаем суммарную длительность аудиозаписей для распознавания по дням в минутах.

In [6]:
daily_duration = unique_files.groupby('date_d')['duration'].sum().reset_index()
daily_duration.duration = daily_duration.duration / 60

In [7]:
daily_duration

,date_d,duration
0,2024-01-01,39.969667
1,2024-01-02,13928.201237
2,2024-01-03,17771.537604
3,2024-01-04,19107.857633
4,2024-01-05,20873.734483
5,2024-01-06,18026.749338
6,2024-01-07,14085.537487
7,2024-01-08,19878.013375
8,2024-01-09,30633.502542
9,2024-01-10,31820.098371


In [14]:
fig = px.line(
    daily_duration,
    x='date_d',
    y='duration',
    markers=True,
    width=1400,
    height=600
).update_layout(
    xaxis_title='Дата',
    yaxis_title='Кол-во минут',
    title=dict(
        text='Длительность записей в минутах',
        x=.5
        )
)
fig.show()

При подсчётете медианного значения не будем учитывать выброс за 1 января.

In [17]:
print(
    'Медианная длительность файлов для расшифровки —',
    f'{round(daily_duration[1:].duration.median())} мин/день'
)

Медианная длительность файлов для расшифровки — 20376 мин/день
